In [2]:
import cv2
import os
import numpy as np

import tensorflow as tf
import keras
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer, Flatten, Dense
from sklearn.model_selection import train_test_split

from model import MBConv, SEBlock


ModuleNotFoundError: No module named 'cv2'

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
train_path = 'train'
labels = os.listdir(train_path)
X = []
y = []
for label in labels:
    path = os.path.join(train_path, label)
    image_names = os.listdir(path)

    for image_name in image_names:
        color_img = cv2.imread(os.path.join(path, image_name))
        gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
        processed = cv2.normalize(gray_img, None, 0, 255, cv2.NORM_MINMAX)
        img = np.concatenate([color_img, processed[:, :, np.newaxis]], axis=2)
        X.append(img)
        y.append(label)
y = list(map(int, y))

In [23]:
print(len(X), len(y))

451 451


In [8]:
X[0].shape

(250, 250, 4)

In [24]:
one_hot = tf.one_hot(y, 3)
one_hot.shape

TensorShape([451, 3])

In [6]:
batch_size = 64

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(X, one_hot, test_size=0.1, random_state=568)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([322, 439, 198,  45,  12, 269, 137, 318, 365,  11, 409, 242, 450,
       411,  64, 264,  99, 449, 327, 288, 345, 296, 321, 354, 213,  75,
       131,  43,   6,  28, 426, 370,  13, 111, 180, 243, 248, 375, 372,
       185, 208, 319, 228, 419,  10, 362, 400, 220,  26, 315, 387, 206,
       193, 323,  84, 309,  83, 393,  49, 112, 122,  35, 145, 233, 133,
       100, 313, 151,  55, 286, 110, 114, 421, 271, 408, 305, 297, 200,
       418,   3, 390,  31, 282, 440,  91, 134, 175, 363, 424,  81, 300,
        93,  71,  95, 340,  70, 235, 116, 381, 171, 416, 268, 328, 234,
       214, 236, 348, 189,  54, 261, 105, 405, 197, 285, 401, 164, 427,
       161,  17, 325, 203,  98, 207, 266,  85, 256,  29,  59, 434, 446,
        48, 265, 210,  60, 415, 199, 438,  88,  36, 357,  37,  25, 352,
       389, 402,   7, 312,  74, 124, 379, 192, 378,  38, 107, 143, 310

In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(250, 250, 4)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu", input_shape=(250, 250, 4)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu", input_shape=(250, 250, 4)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 29, 29, 64)       0

In [13]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(
    from_logits=True), metrics=['accuracy'])


In [ ]:
history = model.fit()